# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [24]:
import pandas as pd
import pymongo

In [25]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [26]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


## Question 0

### Netoyer les données

In [27]:
df_youtube.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7992 entries, 0 to 7991
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   video_id        7992 non-null   object 
 1   title           7992 non-null   object 
 2   channel_title   7992 non-null   object 
 3   category_id     7992 non-null   int64  
 4   tags            7992 non-null   object 
 5   views           7992 non-null   int64  
 6   likes           7992 non-null   int64  
 7   dislikes        7992 non-null   int64  
 8   comment_total   7992 non-null   int64  
 9   thumbnail_link  7992 non-null   object 
 10  date            7992 non-null   float64
dtypes: float64(1), int64(5), object(5)
memory usage: 686.9+ KB


In [28]:
import json
import pandas as pd

with open("data/US_category_id.json", "r") as f:
    json_youtube = json.load(f)

print(json_youtube.keys())  

df_categories = pd.DataFrame(json_youtube["items"])

# Extraire la colonne 'snippet' et création nouveau df
df_categories = pd.concat([
    df_categories.drop("snippet", axis=1),  
    df_categories["snippet"].apply(pd.Series)  
], axis=1)

print(df_categories.head())

if df_youtube.isna().any().any(): 
    print("Il y a des valeurs manquantes dans le tableau.")
else:
    print("Il n'y a pas de valeurs manquantes dans le tableau.")

df_youtube["video_id"] = df_youtube["video_id"].astype(str)
df_youtube["title"] = df_youtube["title"].astype(str)
df_youtube["channel_title"] = df_youtube["channel_title"].astype(str)
df_youtube["tags"] = df_youtube["tags"].astype(str)
df_youtube["thumbnail_link"] = df_youtube["thumbnail_link"].astype(str)
df_youtube["category_id"] = df_youtube["category_id"].astype(str)

df_cleaned = pd.merge(df_youtube, df_categories, left_on="category_id", right_on="id", how="left")

df_cleaned.head()


dict_keys(['kind', 'etag', 'items'])
                    kind                                               etag  \
0  youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...   
1  youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...   
2  youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...   
3  youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...   
4  youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...   

   id                 channelId             title  assignable  
0   1  UCBR8-60-B28hp2BmDPdntcQ  Film & Animation        True  
1   2  UCBR8-60-B28hp2BmDPdntcQ  Autos & Vehicles        True  
2  10  UCBR8-60-B28hp2BmDPdntcQ             Music        True  
3  15  UCBR8-60-B28hp2BmDPdntcQ    Pets & Animals        True  
4  17  UCBR8-60-B28hp2BmDPdntcQ            Sports        True  
Il n'y a pas de valeurs manquantes dans le tableau.


,video_id,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,kind,etag,id,channelId,title_y,assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",24,UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",28,UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...",22,UCBR8-60-B28hp2BmDPdntcQ,People & Blogs,True
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",28,UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...",23,UCBR8-60-B28hp2BmDPdntcQ,Comedy,True


In [29]:
df_cleaned = df_cleaned.rename(columns={"title_y": "category_title"})
df_cleaned.drop("id", axis=1, inplace=True)
df_cleaned.drop("category_id", axis=1, inplace=True)
df_cleaned.drop("kind", axis=1, inplace=True)
df_cleaned.drop("channelId", axis=1, inplace=True)
df_cleaned.drop("etag", axis=1, inplace=True)
df_cleaned.drop("assignable", axis=1, inplace=True)

df_cleaned.head()

,video_id,title_x,channel_title,tags,views,likes,dislikes,comment_total,thumbnail_link,date,category_title
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,Entertainment
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,Science & Technology
2,cLdxuaxaQwc,My Response,PewDiePie,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09,People & Blogs
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09,Science & Technology
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09,Comedy


### Importer les données

In [35]:
client.drop_database('exercices')

client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

# Conversion DataFrame -> Liste de dictionnaires
data = df_cleaned.to_dict(orient="records")

# Insertion des données
collection.insert_many(data)

print("Données insérées avec succès !")


Données insérées avec succès !


In [36]:
print(client.list_database_names())
print(database.list_collection_names())

['admin', 'config', 'exercices', 'local']
['youtube']


## Question 1  

In [37]:
# Récupérer toutes les vidéos de la chaîne Apple.

apple_video = collection.find({"channel_title": "Apple"})

for video in apple_video:
    print(video)

{'_id': ObjectId('676184735db168695d5005f8'), 'video_id': 'K4wEI5zhHB0', 'title_x': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone', 'views': 7860119, 'likes': 185853, 'dislikes': 26679, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': 13.09, 'category_title': 'Science & Technology'}
{'_id': ObjectId('676184735db168695d5006c2'), 'video_id': 'K4wEI5zhHB0', 'title_x': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone', 'views': 12200526, 'likes': 258842, 'dislikes': 44339, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': 14.09, 'category_title': 'Science & Technology'}
{'_id': Obj

## Question 2

In [38]:
# Compter le nombre de catégories différentes

# Utiliser distinct pour récupérer les catégories uniques
categories = collection.distinct("category_title")  

# Compter le nombre de catégories distinctes
num_categories = len(categories)

print(f"Nombre de catégories différentes : {num_categories}")

Nombre de catégories différentes : 16


## Question 3

In [39]:
# Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update

# Découper les tags en liste et mettre à jour chaque document
videos = collection.find()  # Récupère tous les documents de la collection

for video in videos:
    # Découper les tags en liste, en utilisant le séparateur '|' (ou un autre si nécessaire)
    tags_list = video.get("tags", "").split("|")
    
    # Mettre à jour le document avec la liste de tags
    collection.update_one(
        {"_id": video["_id"]},  # Filtre pour trouver le document spécifique
        {"$set": {"tags": tags_list}}  # Mise à jour de la clé 'tags' avec la liste
    )

print("Tags mis à jour avec succès !")


Tags mis à jour avec succès !


## Question 4

In [41]:
# Récupérer les vidéos les plus vues

most_viewed_videos = collection.find().sort('views', pymongo.DESCENDING).limit(20)

for video in most_viewed_videos:
    print(f"Vidéo: {video['title_x']} | Vues: {video['views']}")

Vidéo: BTS (방탄소년단) 'DNA' Official MV | Vues: 41500672
Vidéo: BTS (방탄소년단) 'DNA' Official MV | Vues: 38013692
Vidéo: ZAYN - Dusk Till Dawn ft. Sia | Vues: 36323498
Vidéo: Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher | Vues: 33191594
Vidéo: Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher | Vues: 32512343
Vidéo: Shakira - Perro Fiel (Official Video) ft. Nicky Jam | Vues: 32136948
Vidéo: Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher | Vues: 31636010
Vidéo: BTS (방탄소년단) 'DNA' Official MV | Vues: 30424471
Vidéo: Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher | Vues: 30224787
Vidéo: Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher | Vues: 28212358
Vidéo: Star Wars: The Last Jedi Trailer (Official) | Vues: 27909589
Vidéo: Shakira - Perro Fiel (Official Video) ft. Nicky Jam | Vues: 27750173
Vidéo: Star Wars: The Last Jedi Trailer (Official) | Vues: 26208268
Vidéo: Eminem Rips Donald Trump In BET Hip Hop Awards Fr

## Question 5

In [42]:
# Compter le nombre moyen de vues en fonction de la catégorie

# Récupérer les vidéos et calculer la moyenne des vues par catégorie
pipeline = [
    {
        "$group": {
            "_id": "$category_title",  # Grouper par catégorie
            "average_views": {"$avg": "$views"}  # Calculer la moyenne des vues
        }
    },
    {
        "$sort": {"average_views": -1}  # Optionnel : trier par moyenne de vues, du plus grand au plus petit
    }
]

# Exécuter le pipeline d'agrégation
category_avg_views = collection.aggregate(pipeline)

# Afficher les résultats
for category in category_avg_views:
    print(f"Catégorie: {category['_id']} | Moyenne des vues: {category['average_views']}")


Catégorie: Comedy | Moyenne des vues: 1240073.294039735
Catégorie: Music | Moyenne des vues: 1176553.612
Catégorie: Entertainment | Moyenne des vues: 1154868.4397251718
Catégorie: Nonprofits & Activism | Moyenne des vues: 1110334.2142857143
Catégorie: Film & Animation | Moyenne des vues: 1039472.6560846561
Catégorie: People & Blogs | Moyenne des vues: 971532.7573696146
Catégorie: Science & Technology | Moyenne des vues: 924730.548828125
Catégorie: Sports | Moyenne des vues: 728434.3243902439
Catégorie: Gaming | Moyenne des vues: 681081.8170731707
Catégorie: Pets & Animals | Moyenne des vues: 651404.3879310344
Catégorie: Autos & Vehicles | Moyenne des vues: 607693.8706896552
Catégorie: Education | Moyenne des vues: 547582.4700598803
Catégorie: News & Politics | Moyenne des vues: 540955.5569823436
Catégorie: Howto & Style | Moyenne des vues: 537665.1806674339
Catégorie: Travel & Events | Moyenne des vues: 464041.0833333333
Catégorie: Shows | Moyenne des vues: 8492.5


## Question 6 

In [43]:
# Récupérer les chaines Youtube avec la plus grande moyenne de likes

# Récupérer les vidéos et calculer la moyenne des likes par chaîne
pipeline = [
    {
        "$group": {
            "_id": "$channel_title",  # Grouper par chaîne
            "average_likes": {"$avg": "$likes"}  # Calculer la moyenne des likes
        }
    },
    {
        "$sort": {"average_likes": -1}  # Trier par moyenne des likes, du plus grand au plus petit
    }
]

# Exécuter le pipeline d'agrégation
channel_avg_likes = collection.aggregate(pipeline)

# Afficher les résultats
for channel in channel_avg_likes:
    print(f"Chaîne: {channel['_id']} | Moyenne des likes: {channel['average_likes']}")

Chaîne: ZaynVEVO | Moyenne des likes: 1431683.0
Chaîne: ibighit | Moyenne des likes: 1371766.25
Chaîne: melanie martinez | Moyenne des likes: 911871.5
Chaîne: BETNetworks | Moyenne des likes: 769687.625
Chaîne: jypentertainment | Moyenne des likes: 758826.6666666666
Chaîne: Logan Paul Vlogs | Moyenne des likes: 653443.5384615385
Chaîne: Wengie | Moyenne des likes: 567235.2
Chaîne: Alesso | Moyenne des likes: 509417.28571428574
Chaîne: PewDiePie | Moyenne des likes: 508647.8
Chaîne: shakiraVEVO | Moyenne des likes: 434302.14285714284
Chaîne: Ed Sheeran | Moyenne des likes: 395654.1666666667
Chaîne: Dude Perfect | Moyenne des likes: 389697.1111111111
Chaîne: Conor Maynard | Moyenne des likes: 389273.0
Chaîne: TaylorSwiftVEVO | Moyenne des likes: 367455.6666666667
Chaîne: Liza Koshy | Moyenne des likes: 365146.9583333333
Chaîne: Linkin Park | Moyenne des likes: 356475.7272727273
Chaîne: jbalvinVEVO | Moyenne des likes: 345130.4285714286
Chaîne: Primitive Technology | Moyenne des likes: 34